In [8]:
!pip install streamlit pandas plotly
!pip install rpy2


In [9]:
# Import rpy2 for running R code
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

# Enable pandas <-> R data frame conversion
pandas2ri.activate()

# Install R packages if not already installed
ro.r('if (!requireNamespace("tidyverse", quietly = TRUE)) install.packages("tidyverse")')
ro.r('if (!requireNamespace("lubridate", quietly = TRUE)) install.packages("lubridate")')
ro.r('if (!requireNamespace("TTR", quietly = TRUE)) install.packages("TTR")')
ro.r('if (!requireNamespace("PerformanceAnalytics", quietly = TRUE)) install.packages("PerformanceAnalytics")')
ro.r('if (!requireNamespace("xts", quietly = TRUE)) install.packages("xts")')


# R packages
ro.r('library(tidyverse)')
ro.r('library(lubridate)')
ro.r('library(TTR)')
ro.r('library(PerformanceAnalytics)')
ro.r('library(xts)')

In [10]:
import streamlit as st
import pandas as pd
import plotly.express as px
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

pandas2ri.activate()

st.title("Parabolic SAR Backtest Dashboard")

# 1. CSV Upload
uploaded_file = st.file_uploader("Upload your CSV (datetime, open, high, low, close, volume)", type=["csv"])
if uploaded_file is not None:
    df = pd.read_csv(uploaded_file)
    st.write("Preview of your data:")
    st.dataframe(df.head())

    # 2. SAR Parameters
    accel = st.number_input("SAR Acceleration", min_value=0.001, max_value=0.1, value=0.005, step=0.001)
    max_accel = st.number_input("SAR Maximum", min_value=0.01, max_value=0.5, value=0.05, step=0.01)

    if st.button("Run Backtest"):
        st.info("Running backtest in R...")

        # Pass CSV to R
        ro.globalenv['df_prices'] = pandas2ri.py2rpy(df)
        ro.globalenv['accel'] = accel
        ro.globalenv['max_accel'] = max_accel

        # R code as a multi-line string
        r_code = """
        df_prices <- df_prices %>%
          mutate(datetime = lubridate::dmy_hm(datetime)) %>%
          arrange(datetime) %>%
          filter(datetime >= lubridate::ymd("2021-01-01"))

        df_prices <- df_prices %>%
          mutate(
            sar = TTR::SAR(cbind(high, low), accel=c(accel, accel)),
            signal = dplyr::case_when(
              close > sar ~ 1,
              close < sar ~ -1,
              TRUE ~ 0
            )
          )

        initial_capital <- 100000
        transaction_cost <- 100
        df_prices <- df_prices %>%
          mutate(position = dplyr::lag(signal, default=0))

        df_prices <- df_prices %>%
          mutate(
            trade_return = dplyr::case_when(
              position==1 ~ (close/lag(close)-1),
              position==-1 ~ (lag(close)/close-1),
              TRUE ~ 0
            ),
            trade_pnl = trade_return * initial_capital,
            trade_cost = if_else(position != dplyr::lag(position, default=0) & position !=0,
                                 transaction_cost, 0),
            net_pnl = trade_pnl - trade_cost,
            equity = cumsum(net_pnl) + initial_capital
          ) %>% tidyr::replace_na(list(net_pnl=0, equity=initial_capital))

        # Summary Metrics
        daily_returns <- df_prices %>%
          group_by(date=as.Date(datetime)) %>%
          summarise(daily_pnl=sum(net_pnl), equity=last(equity), .groups="drop") %>%
          mutate(daily_ret = daily_pnl / dplyr::lag(equity, default=initial_capital)) %>% tidyr::drop_na()
        daily_xts <- xts::xts(daily_returns$daily_ret, order.by=daily_returns$date)
        sharpe <- PerformanceAnalytics::SharpeRatio.annualized(daily_xts, scale=252, Rf=0)
        max_dd <- PerformanceAnalytics::maxDrawdown(daily_xts)
        summary_metrics <- tibble::tibble(
          Final_Equity = last(df_prices$equity),
          Total_Return = (last(df_prices$equity)/initial_capital-1)*100,
          Sharpe = as.numeric(sharpe),
          Max_Drawdown = as.numeric(max_dd)
        )
        """

        # Run R code
        ro.r(r_code)

        # Get summary metrics back to Python
        summary_metrics = ro.r('summary_metrics')
        summary_df = pandas2ri.rpy2py(summary_metrics)
        st.write("Summary Metrics:")
        st.dataframe(summary_df)

        # Get equity for plot
        equity_df = ro.r('df_prices[, c("datetime","equity")]')
        equity_df = pandas2ri.rpy2py(equity_df)
        equity_df['datetime'] = pd.to_datetime(equity_df['datetime'])
        fig = px.line(equity_df, x='datetime', y='equity', title="Equity Curve")
        st.plotly_chart(fig)


2025-09-16 13:37:23.043 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-16 13:37:23.044 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-16 13:37:23.045 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-16 13:37:23.046 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-16 13:37:23.049 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-16 13:37:23.050 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-16 13:37:23.051 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-16 13:37:23.052 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar